# Libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from ast import literal_eval

# Data Loading

In [2]:
file_path = 'data.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
df.columns

Index(['Long', 'Lat', 'Size', 'Light', 'Price_Booking1', 'Price_Booking2',
       'Price_Booking3', 'Price_Booking4', 'Price_Booking5', 'Price_Booking6',
       ...
       'Price_Booking511', 'Price_Booking512', 'Price_Booking513',
       'Price_Booking514', 'Price_Booking515', 'Price_Booking516',
       'Price_Booking517', 'Price_Booking518', 'Price_Booking519',
       'Price_Booking520'],
      dtype='object', length=524)

Price_Booking values are not suitable to use as tuples I separated values price and booking_ratio

In [5]:
average_prices = {}
booking_ratios = {}
weekly_revenues = {}

for i in range(1, 521):
    column_name = f'Price_Booking{i}'
    # Convert strings to tuples if they are not already
    df[column_name] = df[column_name].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)

    # Prepare new columns separately
    average_prices[f'average_price{i}'] = df[column_name].apply(lambda x: x[0])
    booking_ratios[f'booking_ratio{i}'] = df[column_name].apply(lambda x: x[1])
    weekly_revenues[f'weekly_revenue{i}'] = df[column_name].apply(lambda x: x[0] * x[1] * 7)
    df.drop(columns=f'Price_Booking{i}', inplace=True, axis=1)
# Convert dictionaries to DataFrames
average_price_df = pd.DataFrame(average_prices)
booking_ratio_df = pd.DataFrame(booking_ratios)
weekly_revenue_df = pd.DataFrame(weekly_revenues)

# Concatenate all new DataFrames at once
df = pd.concat([df, weekly_revenue_df, average_price_df, booking_ratio_df], axis=1)

df = df.copy()

In [6]:
df.head()

,Long,Lat,Size,Light,weekly_revenue1,weekly_revenue2,weekly_revenue3,weekly_revenue4,weekly_revenue5,weekly_revenue6,...,booking_ratio511,booking_ratio512,booking_ratio513,booking_ratio514,booking_ratio515,booking_ratio516,booking_ratio517,booking_ratio518,booking_ratio519,booking_ratio520
0,5,-54,medium,0,135.901999,69.306219,139.773848,112.227282,149.083474,80.869084,...,0.8098,0.1080,0.4888,0.0577,0.0434,0.0348,0.2090,0.8847,0.2190,0.2070
1,15,34,large,1,18.695317,57.786859,74.887266,99.041069,11.027450,27.723354,...,0.2932,0.0003,0.0026,0.1442,0.5035,0.0004,0.8583,0.4480,0.9208,0.0792
2,52,33,small,1,44.347306,47.109034,28.251371,3.330332,66.354832,104.397793,...,0.3989,0.8718,0.0075,0.0959,0.2264,0.0155,0.0374,0.4942,0.9249,0.0103
3,36,-32,small,0,180.714657,123.163511,180.788020,87.430023,67.247760,113.274359,...,0.3075,0.4246,0.8802,0.0830,0.3801,0.5986,0.8888,0.0999,0.8736,0.4038
4,29,49,medium,0,57.936356,159.117393,164.657600,149.429804,63.293161,79.321350,...,0.8544,0.5271,0.2676,0.0898,0.0857,0.5754,0.8676,0.3303,0.2868,0.3759


In [8]:
file_path = 'weekly_rev.csv'

In [9]:
df.to_csv(file_path, index=False)

In [10]:
data = pd.read_csv(file_path)

In [11]:
data.columns

Index(['Long', 'Lat', 'Size', 'Light', 'weekly_revenue1', 'weekly_revenue2',
       'weekly_revenue3', 'weekly_revenue4', 'weekly_revenue5',
       'weekly_revenue6',
       ...
       'booking_ratio511', 'booking_ratio512', 'booking_ratio513',
       'booking_ratio514', 'booking_ratio515', 'booking_ratio516',
       'booking_ratio517', 'booking_ratio518', 'booking_ratio519',
       'booking_ratio520'],
      dtype='object', length=1564)

# Maximal revenue prediction

If revune is maximal -> price is optimal

In [12]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, Concatenate, Dropout, Bidirectional

# Preparing the data
size_dummies = pd.get_dummies(data['Size'], prefix='Size')
static_features = data[['Long', 'Lat', 'Light']].join(size_dummies)
scaler = StandardScaler()
static_features = scaler.fit_transform(static_features)

# Preparing revenue data and repeating static features for each time step
revenue_data = data.filter(regex='weekly_revenue').values
revenue_data = revenue_data.reshape(revenue_data.shape[0], revenue_data.shape[1], 1)
static_features_repeated = np.repeat(static_features[:, np.newaxis, :], revenue_data.shape[1], axis=1)

# Combine revenue data with static features
combined_features = np.concatenate([revenue_data, static_features_repeated], axis=-1)
X_train, X_test, y_train, y_test = train_test_split(combined_features, revenue_data[:, -1, 0], test_size=0.2, random_state=42)

I used an LSTM model (RNN NN) for this project because time series data needs to be remembered, which can help in making more accurate predictions

In [13]:
input_shape = combined_features.shape[1:]  # Should be (520, number of features + 1)
model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=input_shape),
    LSTM(50),
    Dense(1)
])

# Compile the model
model_lstm.compile(optimizer='adam', loss='mse')

C:\Users\tatev\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
model_lstm.fit(X_train, y_train, epochs=85, batch_size=32, validation_split=0.1)

# Predict the revenue for week 521
week_521_revenue1 = model_lstm.predict(X_test)

Epoch 1/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 40s 166ms/step - loss: 7415.4536 - val_loss: 5816.3105
Epoch 2/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 40s 177ms/step - loss: 5638.9385 - val_loss: 4640.2622
Epoch 3/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 42s 185ms/step - loss: 4555.4214 - val_loss: 3731.8296
Epoch 4/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 42s 185ms/step - loss: 3523.3674 - val_loss: 3010.7793
Epoch 5/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 40s 179ms/step - loss: 2878.4146 - val_loss: 2433.7693
Epoch 6/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 39s 172ms/step - loss: 2354.6150 - val_loss: 1974.9464
Epoch 7/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 39s 175ms/step - loss: 1887.7947 - val_loss: 1605.6930
Epoch 8/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 40s 176ms/step - loss: 1538.5577 - val_loss: 1306.9663
Epoch 9/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 43s 192ms/step - loss: 1286.6083 - val_loss: 1067.2828
Epoch 10/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 41s 180ms/step - loss: 1016.5706 - val_loss: 873.9867
Epoch 11/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 171ms

225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 170ms/step - loss: 0.4126 - val_loss: 0.4685
Epoch 67/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 169ms/step - loss: 0.2941 - val_loss: 0.4573
Epoch 68/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 167ms/step - loss: 0.5402 - val_loss: 0.3922
Epoch 69/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 167ms/step - loss: 0.3677 - val_loss: 0.5710
Epoch 70/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 168ms/step - loss: 0.4699 - val_loss: 0.3509
Epoch 71/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 167ms/step - loss: 0.3229 - val_loss: 0.2923
Epoch 72/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 37s 166ms/step - loss: 0.4024 - val_loss: 0.4855
Epoch 73/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 37s 166ms/step - loss: 0.3908 - val_loss: 0.3314
Epoch 74/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 167ms/step - loss: 0.2598 - val_loss: 0.2109
Epoch 75/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 169ms/step - loss: 0.2411 - val_loss: 0.4541
Epoch 76/85
225/225 ━━━━━━━━━━━━━━━━━━━━ 38s 167ms/step - loss: 0.4534 - val_loss: 0.3649
Epoch 77/85
225/225 ━━

In [15]:
week_521_revenue1 = model_lstm.predict(combined_features)

313/313 ━━━━━━━━━━━━━━━━━━━━ 18s 58ms/step


In [16]:
week_521_revenue1

array([[ 97.029945],
       [ 30.134897],
       [  6.605833],
       ...,
       [145.59639 ],
       [120.92972 ],
       [107.028854]], dtype=float32)

In [ ]:
data['weekly_revenue521'] = week_521_revenue1

In [21]:
df_bench = pd.read_csv('benchmark_data.csv')

In [22]:
df_bench

,Long,Lat,Size,Light,Price_Booking1,Price_Booking2,Price_Booking3,Price_Booking4,Price_Booking5,Price_Booking6,...,Price_Booking511,Price_Booking512,Price_Booking513,Price_Booking514,Price_Booking515,Price_Booking516,Price_Booking517,Price_Booking518,Price_Booking519,Price_Booking520
0,11,-8,small,1,"(44.07422740778878, 0.2244)","(10.235745495054404, 0.8771)","(77.44432192100861, 0.0115)","(32.682854325706636, 0.4374)","(93.07005020369597, 0.0016)","(59.55342009850416, 0.0712)",...,"(3.503511792465819, 0.8693)","(21.207831688891897, 0.6324)","(37.472733112480896, 0.3145)","(17.68477270798783, 0.7076)","(3.9349006656278496, 0.8869)","(90.75592780176181, 0.0021)","(57.12871533905361, 0.082)","(48.73808820675718, 0.1592)","(12.213765932305831, 0.8287)","(50.82865199262406, 0.1387)"
1,-23,50,large,0,"(27.403085749044997, 0.6838)","(67.88625315230517, 0.1343)","(58.481870400767434, 0.224)","(56.95623287755157, 0.2431)","(4.993203977397098, 0.9579)","(42.47182196384385, 0.4534)",...,"(21.379118386361394, 0.7227)","(58.91370302091187, 0.2023)","(4.511084912965702, 0.8782)","(49.94310488165724, 0.3095)","(11.68847681410703, 0.8466)","(69.85385620221606, 0.1142)","(30.52089300740838, 0.6134)","(72.41102332280923, 0.0994)","(39.81837685219315, 0.4723)","(88.06567678260058, 0.0348)"
2,10,-45,medium,0,"(32.49922734531039, 0.6368)","(27.27239937360443, 0.7202)","(36.20418901494742, 0.5888)","(81.40103513141597, 0.0835)","(71.81075706852077, 0.1446)","(18.065003797032787, 0.8636)",...,"(17.42772963166037, 0.7843)","(7.484976054849901, 0.8631)","(25.601585429981, 0.6956)","(50.3850166054486, 0.3494)","(16.117362334552887, 0.8151)","(14.195214120071231, 0.8388)","(44.23375456195849, 0.4425)","(83.32192167215457, 0.0709)","(54.16402788999447, 0.3136)","(25.087128491645018, 0.7394)"
3,-4,3,small,1,"(92.82272915578096, 0.0011)","(24.132065502164757, 0.6006)","(63.09560763988213, 0.0423)","(33.327256606541425, 0.403)","(33.69975089548419, 0.3979)","(73.18542895745563, 0.0147)",...,"(69.74082104797938, 0.0195)","(38.80884537040286, 0.271)","(63.16126548911956, 0.039)","(88.27001407634896, 0.002)","(74.70630932807055, 0.0114)","(92.17872999659934, 0.0012)","(97.04303626476803, 0.0006)","(2.4528886659984206, 0.9133)","(59.1714243644762, 0.0596)","(61.30086613145307, 0.0492)"
4,-2,15,small,1,"(49.80177382910645, 0.1483)","(54.07334498175166, 0.1074)","(45.903234713953736, 0.1989)","(77.4330848736851, 0.0111)","(65.96155090109079, 0.0379)","(5.084813285843259, 0.9554)",...,"(66.97356607076182, 0.0311)","(91.1566259740697, 0.0018)","(85.68419936773684, 0.0037)","(75.55400762308903, 0.0127)","(48.515525854018584, 0.1555)","(60.91412796408466, 0.057)","(39.00983918378642, 0.2932)","(19.47406031706128, 0.6919)","(94.582862203968, 0.0012)","(35.19505761977505, 0.3708)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,54,46,large,0,"(38.92010905514978, 0.6391)","(43.67199655852267, 0.5828)","(79.3108073425907, 0.1922)","(33.835143318405905, 0.7183)","(8.094740680730572, 0.952)","(30.051272039367525, 0.7738)",...,"(16.501405960541682, 0.8184)","(48.548879030391035, 0.4833)","(92.11811494885823, 0.1019)","(25.932470821018295, 0.7508)","(55.42036821166823, 0.4102)","(57.53425442297285, 0.3887)","(50.4710198315531, 0.4757)","(59.57297892738654, 0.3715)","(52.41657965896061, 0.4593)","(84.69875384380687, 0.1499)"
1996,-33,30,small,1,"(63.414821080339145, 0.1132)","(96.5251344234855, 0.007)","(48.693655619087295, 0.274)","(83.48267652523833, 0.0245)","(38.41139224623771, 0.4452)","(35.417629666876074, 0.5032)",...,"(16.825458038755116, 0.7557)","(99.07768920144888, 0.005)","(36.36184221758047, 0.4416)","(86.57729771540193, 0.0172)","(35.933910870024555, 0.4546)","(94.78582147599467, 0.008)","(42.88209797584985, 0.3458)","(87.63162373682749, 0.0161)","(74.36260170592735, 0.0503)","(48.352661829682454, 0.2723)"
1997,32,-44,large,1,"(78.75671906763031, 0.0079)","(85.83768213401827, 0.0032)","(94.06346122190314, 0.001)","(2

In [23]:
average_prices_b = {}
booking_ratios_b = {}
weekly_revenues_b = {}

for i in range(1, 521):
    column_name = f'Price_Booking{i}'
    # Convert strings to tuples if they are not already
    df_bench[column_name] = df_bench[column_name].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)

    # Prepare new columns separately
    average_prices_b[f'average_price{i}'] = df_bench[column_name].apply(lambda x: x[0])
    booking_ratios_b[f'booking_ratio{i}'] = df_bench[column_name].apply(lambda x: x[1])
    weekly_revenues_b[f'weekly_revenue{i}'] = df_bench[column_name].apply(lambda x: x[0] * x[1] * 7)
    df_bench.drop(columns=f'Price_Booking{i}', inplace=True, axis=1)
# Convert dictionaries to DataFrames
average_price_bdf = pd.DataFrame(average_prices_b)
booking_ratio_bdf = pd.DataFrame(booking_ratios_b)
weekly_revenue_bdf = pd.DataFrame(weekly_revenues_b)

# Concatenate all new DataFrames at once
df_bench = pd.concat([df_bench, weekly_revenue_bdf, average_price_bdf, booking_ratio_bdf], axis=1)

df_bench = df_bench.copy()

In [24]:
df_bench.head()

,Long,Lat,Size,Light,weekly_revenue1,weekly_revenue2,weekly_revenue3,weekly_revenue4,weekly_revenue5,weekly_revenue6,...,booking_ratio511,booking_ratio512,booking_ratio513,booking_ratio514,booking_ratio515,booking_ratio516,booking_ratio517,booking_ratio518,booking_ratio519,booking_ratio520
0,11,-8,small,1,69.231796,62.844407,6.234268,100.068363,1.042385,29.681425,...,0.8693,0.6324,0.3145,0.7076,0.8869,0.0021,0.0820,0.1592,0.8287,0.1387
1,-23,50,large,0,131.167610,63.819867,91.699573,96.922421,33.480931,134.797069,...,0.7227,0.2023,0.8782,0.3095,0.8466,0.1142,0.6134,0.0994,0.4723,0.0348
2,10,-45,medium,0,144.868556,137.491074,149.219185,47.578905,72.686848,109.206561,...,0.7843,0.8631,0.6956,0.3494,0.8151,0.8388,0.4425,0.0709,0.3136,0.7394
3,-4,3,small,1,0.714735,101.456030,18.682609,94.016191,93.863916,7.530781,...,0.0195,0.2710,0.0390,0.0020,0.0114,0.0012,0.0006,0.9133,0.0596,0.0492
4,-2,15,small,1,51.699221,40.652341,63.911074,6.016551,17.499599,34.006214,...,0.0311,0.0018,0.0037,0.0127,0.1555,0.0570,0.2932,0.6919,0.0012,0.3708


In [26]:
size_dummies_b = pd.get_dummies(df_bench['Size'], prefix='Size')
static_features_b = df_bench[['Long', 'Lat', 'Light']].join(size_dummies_b)
scaler = StandardScaler()
static_features_b = scaler.fit_transform(static_features_b)

# Preparing revenue data and repeating static features for each time step
revenue_data_b = df_bench.filter(regex='weekly_revenue').values
revenue_data_b = revenue_data_b.reshape(revenue_data_b.shape[0], revenue_data_b.shape[1], 1)
static_features_repeated_b = np.repeat(static_features_b[:, np.newaxis, :], revenue_data_b.shape[1], axis=1)

# Combine revenue data with static features
combined_features = np.concatenate([revenue_data_b, static_features_repeated_b], axis=-1)

Model trained for data used to predict benchmark weekly_revenue521

In [27]:
week_521_revenue_b = model_lstm.predict(combined_features)

63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step


In [30]:
week_521_revenue_b

array([[ 49.14397],
       [ 21.7006 ],
       [129.89731],
       ...,
       [ 73.75077],
       [117.18138],
       [132.00987]], dtype=float32)

In [37]:
df_bench['weekly_recenue521'] = week_521_revenue_b

In [38]:
df_bench['weekly_recenue521']

0        49.143970
1        21.700600
2       129.897308
3        20.968540
4        91.287262
           ...    
1995     88.906555
1996     92.031754
1997     73.750771
1998    117.181381
1999    132.009872
Name: weekly_recenue521, Length: 2000, dtype: float32

# Weekly Average Price and Booking Ratio relation

In [39]:
filtered_df = df_bench.filter(regex='^(average_price|booking_ratio)')

In [40]:
filtered_df

,average_price1,average_price2,average_price3,average_price4,average_price5,average_price6,average_price7,average_price8,average_price9,average_price10,...,booking_ratio511,booking_ratio512,booking_ratio513,booking_ratio514,booking_ratio515,booking_ratio516,booking_ratio517,booking_ratio518,booking_ratio519,booking_ratio520
0,44.074227,10.235745,77.444322,32.682854,93.070050,59.553420,8.500513,21.970928,93.906537,86.610440,...,0.8693,0.6324,0.3145,0.7076,0.8869,0.0021,0.0820,0.1592,0.8287,0.1387
1,27.403086,67.886253,58.481870,56.956233,4.993204,42.471822,57.411377,96.337717,5.062147,94.486441,...,0.7227,0.2023,0.8782,0.3095,0.8466,0.1142,0.6134,0.0994,0.4723,0.0348
2,32.499227,27.272399,36.204189,81.401035,71.810757,18.065004,55.501385,14.173636,49.645996,5.737980,...,0.7843,0.8631,0.6956,0.3494,0.8151,0.8388,0.4425,0.0709,0.3136,0.7394
3,92.822729,24.132066,63.095608,33.327257,33.699751,73.185429,98.633290,30.368321,0.787648,26.223341,...,0.0195,0.2710,0.0390,0.0020,0.0114,0.0012,0.0006,0.9133,0.0596,0.0492
4,49.801774,54.073345,45.903235,77.433085,65.961551,5.084813,71.945818,34.142620,12.772324,11.917745,...,0.0311,0.0018,0.0037,0.0127,0.1555,0.0570,0.2932,0.6919,0.0012,0.3708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38.920109,43.671997,79.310807,33.835143,8.094741,30.051272,12.030133,37.424384,14.899707,98.276312,...,0.8184,0.4833,0.1019,0.7508,0.4102,0.3887,0.4757,0.3715,0.4593,0.1499
1996,63.414821,96.525134,48.693656,83.482677,38.411392,35.417630,54.864760,7.573367,3.150291,46.112817,...,0.7557,0.0050,0.4416,0.0172,0.4546,0.0080,0.3458,0.0161,0.0503,0.2723
1997,78.756719,85.837682,94.063461,28.960625,80.025381,88.333021,49.026680,15.772745,90.842968,55.334324,...,0.0356,0.7987,0.4120,0.0029,0.0014,0.6548,0.0006,0.0375,0.0004,0.8214
1998,9.971771,43.748333,76.780431,69.485612,45.632420,20.855097,34.691470,53.437203,80.433463,78.100971,...,0.4467,0.4350,0.8155,0.7813,0.4293,0.7527,0.6186,0.2726,0.7957,0.8210


In [57]:
# Correlation
for i in range(1,521):
  print(filtered_df[f'average_price{i}'].corr(filtered_df[f'booking_ratio{i}']))

-0.9285218255536773
-0.9305331653757585
-0.9308203934142318
-0.928298208494276
-0.9283267981429598
-0.9289976250783964
-0.9302398040035296
-0.9322528401968062
-0.932040107897069
-0.9340990854379891
-0.9303800811675597
-0.9277326329194073
-0.9332739235401228
-0.9319923048764998
-0.9277201765232725
-0.9295565881110057
-0.9358177835380271
-0.9354717308056337
-0.9298857001878821
-0.929487917371072
-0.9324242609450549
-0.9307545891234864
-0.9342944030879619
-0.9324772767143975
-0.9317617441468911
-0.9373686296425866
-0.9296664698363347
-0.9307977130922285
-0.9339789457888036
-0.9282990801117765
-0.9308511773509732
-0.9284485172957134
-0.9298563283121754
-0.9315160908506455
-0.92957510786854
-0.930325506394097
-0.9319652526274573
-0.9316049354087118
-0.9307422054717867
-0.9285128644291386
-0.9352413062899461
-0.9307731488949297
-0.9317852126115314
-0.9293803779055153
-0.9319968508700173
-0.9322907575140782
-0.9349941242818323
-0.9314850183021176
-0.9301492752353111
-0.9325854439601214
-0.928

Correlation between booking_ratio and average price about -0.93, which interpred as strong negative relation

In [58]:
price_columns = [col for col in filtered_df.columns if 'average_price' in col]
ratios = [col for col in filtered_df.columns if 'booking_ratio' in col]

In [ ]:
df_bench.rename(columns={'weekly_revenue521': 'Prices'}, inplace=True)

In [44]:
output_file = 'output_benchmark_data.csv'

There is a strong correlation between price and ratio, so a simple Linear Regression model is used to predict the booking ratio from the average price

Liear Regression trained to predict booking ratio from average price. Then done predictions for weekly_revenue521/7 (It's the highest possible price for specific revenue). Then decrease price to have price*ratio*7 = weekly_revenue equation

In [88]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Define alpha value for Ridge regularization
alpha = 10

# Initialize lists to store evaluation metrics
r2_scores = []
rmse_scores = []
coefs = []

for i in range(filtered_df.shape[0]):
    # Extract prices and ratios
    transposed_prices = filtered_df[price_columns].iloc[i].T
    trans_ratios = filtered_df[ratios].iloc[i].T

    values_array = transposed_prices.to_numpy()
    v_arr = trans_ratios.to_numpy()

    pred_df = pd.DataFrame({
        'Prices': values_array.flatten(),
        'Ratios': v_arr.flatten()
    })

    X = pred_df[['Prices']]
    y = pred_df['Ratios']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize Ridge regression model with specified alpha
    model = Ridge(alpha=alpha)

    # Fit the model
    model.fit(X_train, y_train)

    # Predict ratios for the price from df_bench
    price_start = df_bench[['Prices']].iloc[[i]] / 7
    #print(price_start)
    prediction_ = model.predict(price_start)
    #print(f"Predicted ratio for price: {prediction_[0]} and price is: {price_start}")
    pred_st = prediction_
    st = df_bench[['Prices']].iloc[[i]]
    #print(st)
    x = price_start.copy() 
    #print(f'price_start is {price_start}')
    
    while (x.values * 7 * prediction_ != st.values).all() and (x.values > 0).all():
        x -= 0.1 
        prediction_ = model.predict(x)
        #print(f'Price is: {x} pred ratio is: {prediction_} Start_price is: {price_start.values}')
    t = ()
    if (x.values < 0).all():
        t = (price_start.values[0][0], pred_st[0])
    else:
        t = (x.values[0][0], pred_st[0])
    print(t)
    # Evaluate the model using cross-validation
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    r2_scores_cv = cross_val_score(model, X, y, cv=kf, scoring='r2')
    r2_scores.append(np.mean(r2_scores_cv))
    mse_scores_cv = cross_val_score(model, X, y, cv=kf, scoring=make_scorer(mean_squared_error))
    rmse_scores.append(np.mean(np.sqrt(mse_scores_cv)))
    coefs.append(model.coef_[0])

# Print average evaluation metrics and coefficients
print('Average R²:', np.mean(r2_scores))
print('Average RMSE:', np.mean(rmse_scores))
#print('Coefs:', coefs)

(7.0205674, 0.7600752456441224)
(3.1000857, 0.9119506609448046)
(18.556759, 0.7714866480710033)
(2.9955058, 0.7776505887767543)
(13.041038, 0.6997741699655285)
(2.9212766, 0.929714425659357)
(14.541902, 0.7985111485849041)
(5.412914, 0.7760754168835111)
(14.172444, 0.7882545218337368)
(0.9719432, 0.8671684683183849)
(1.1792558, 0.9133866731805156)
(4.3866625, 0.941858836572866)
(24.654783, 0.7624759792806068)
(1.9987867, 0.8899160054167329)
(13.544943, 0.8072962352438904)
(12.282817, 0.6107094261129532)
(16.554539, 0.8100130516547409)
(0.60571164, 0.8568609914361383)
(12.147621, 0.6218629023297385)
(1.6273702, 0.9439418461637913)
(0.25523123, 0.9951077981156579)
(6.8815875, 0.8351236482242061)
(0.24396825, 0.8152272048209305)
(21.768536, 0.7497908890395438)
(0.29726687, 0.8580407960422383)
(1.7060547, 0.9029333924296321)
(11.397568, 0.8498366701320303)
(2.590768, 0.977212341470524)
(16.730207, 0.7213745264903724)
(17.7598, 0.8098917360231747)
(7.3101683, 0.8096226719941518)
(1.8504927,

(14.111824, 0.7526451540576427)
(6.432472, 0.9299708314348204)
(8.717521, 0.8958681997502431)
(3.0088875, 0.9475644278853842)
(5.0010753, 0.8347449978850185)
(7.9910097, 0.8922515550807391)
(15.866678, 0.6994819371418994)
(1.5412205, 0.876889757320779)
(18.962938, 0.7421610710106731)
(3.2979267, 0.9386654538225241)
(0.6355861, 0.9123723587665211)
(11.499339, 0.8965294915802475)
(1.5670357, 0.7389787031276575)
(20.12183, 0.7881219051899981)
(28.844343, 0.7911998687517214)
(17.68614, 0.8391018357901736)
(4.243194, 0.8730498309046134)
(25.372097, 0.7671252470079346)
(20.463629, 0.7546388418671256)
(0.78022444, 0.8579738929521903)
(0.6647808, 0.9138059661194544)
(19.407675, 0.7544424439002294)
(3.924158, 0.9173676404184548)
(16.496593, 0.8248973044279051)
(2.9694078, 0.7392270291273402)
(10.3551, 0.7551429203634782)
(13.821711, 0.758894919296403)
(9.5476675, 0.926648212165854)
(12.839073, 0.792577907958845)
(0.43040377, 0.8779170307212711)
(14.526192, 0.6659656921643206)
(16.407465, 0.8063

(20.567312, 0.7712033508840571)
(1.1518724, 0.9662694181079342)
(13.07018, 0.791539784975185)
(11.988237, 0.6301513507805734)
(8.853732, 0.9323726114887951)
(25.30835, 0.7812760240577487)
(2.828558, 0.9386079156342705)
(1.621494, 0.9515014424352237)
(12.396071, 0.6902274689285697)
(17.857954, 0.7540503533351509)
(0.2555979, 0.8382015034920931)
(9.090911, 0.8934678972128911)
(6.8533664, 0.886262605351368)
(10.072411, 0.8946741865085819)
(11.841455, 0.863175562168184)
(22.056494, 0.7490838937782207)
(16.944345, 0.7291252399149252)
(0.8132671, 0.8953161295964547)
(7.29889, 0.9225465193751428)
(11.82627, 0.6166736977306502)
(17.269165, 0.8401429826736205)
(7.1176567, 0.805748418927806)
(12.913606, 0.8504859120814107)
(27.062912, 0.7899750724713817)
(17.64219, 0.7873558407828107)
(14.283051, 0.8400667809637152)
(24.98611, 0.758232775763875)
(8.889357, 0.8953236873226115)
(10.622344, 0.7234757677124936)
(8.528059, 0.9179477989084386)
(1.592933, 0.8426826827201757)
(6.6938066, 0.9454495867771

(10.312797, 0.8318319929024459)
(19.370085, 0.738624547574567)
(1.4881744, 0.8331890844506024)
(25.46384, 0.7596475665638606)
(3.1190865, 0.9463830447779881)
(11.570112, 0.8115567976550584)
(12.962516, 0.6465575139067576)
(1.6061344, 0.8088866703513148)
(0.39128494, 0.7309836025843315)
(14.968195, 0.8092512537433296)
(5.8175993, 0.9316313400714515)
(19.00349, 0.7950805728564596)
(19.644428, 0.735621690517266)
(2.5998495, 0.933173953508711)
(5.9698224, 0.8705297908623)
(8.006176, 0.7665198533800455)
(2.3165207, 0.8221553897611341)
(12.849017, 0.8814778358356785)
(0.24072461, 0.8322173363969114)
(13.826081, 0.6742895146967715)
(21.679722, 0.789927935547486)
(21.053177, 0.7707556575470974)
(18.72851, 0.7945900425140894)
(10.22256, 0.7003339639092308)
(12.322673, 0.6968054063938747)
(4.078172, 0.768203881180295)
(17.29761, 0.7210634588215598)
(19.890848, 0.7810514197475305)
(4.1641088, 0.9388526731999008)
(2.946232, 0.9140448124760492)
(1.9848349, 0.7894268422473657)
(3.679477, 0.804456935

(15.995384, 0.7931863657450289)
(5.296022, 0.83614356321593)
(21.41048, 0.7466354772039775)
(11.209746, 0.7551747562751587)
(0.98489857, 0.9102315890326614)
(26.456411, 0.7604336257450299)
(15.410499, 0.7428346975247211)
(16.150042, 0.7954655286356234)
(3.4855225, 0.8439522403965061)
(18.940216, 0.8167815959369459)
(23.095358, 0.7594044523933299)
(14.49727, 0.8731929492495638)
(6.9899793, 0.8088655806141689)
(6.4606357, 0.9065477289994373)
(0.35768676, 0.8220042176228672)
(10.422233, 0.8022423234556465)
(1.7020285, 0.9365437320577051)
(8.715435, 0.9079412057981178)
(10.299327, 0.9207050709205794)
(15.605471, 0.808305153688139)
(15.730789, 0.8449316654909463)
(16.05258, 0.7770729291470205)
(10.813436, 0.7327942412434303)
(14.749968, 0.7659886132888268)
(18.923616, 0.8293083662013369)
(16.716652, 0.7978321287422372)
(15.663079, 0.7323591379858863)
(14.653575, 0.8591268948957713)
(17.085693, 0.7994657497031841)
(1.8279841, 0.9121169592332254)
(5.8133736, 0.9055881676435948)
(0.24092361, 0

(15.765897, 0.733064119281581)
(2.4119153, 0.8151393305857207)
(24.67103, 0.7759550193598113)
(1.0468403, 0.8009258341679316)
(23.675581, 0.7709093261978945)
(8.530184, 0.8767987704172545)
(5.1245446, 0.9145945287430569)
(11.052005, 0.8534157076506975)
(19.113083, 0.7723359139288074)
(2.9138546, 0.9196447635899649)
(0.32027435, 0.8131567186916389)
(0.47789207, 0.7719891806783685)
(10.890528, 0.6732804650087653)
(4.314717, 0.9285021418498471)
(14.847635, 0.6824256121584621)
(21.879162, 0.7974183779910161)
(19.543009, 0.7541275311709714)
(18.705427, 0.7846355667121856)
(9.757523, 0.8671900863760577)
(21.516285, 0.7482658536136385)
(18.384432, 0.7785685441025701)
(16.971622, 0.7264453213492603)
(0.30578235, 0.7681927462228385)
(2.4651427, 0.9059763472295183)
(22.272034, 0.750423159591376)
(15.478365, 0.686455859862622)
(8.928769, 0.8168189240682474)
(19.741224, 0.7447755734088348)
(0.25319102, 0.8439411031771484)
(16.055418, 0.7091949037241417)
(7.7112164, 0.7839438996595482)
(13.949377, 

(12.989603, 0.6822186477614487)
(4.719528, 0.9859075148972055)
(12.33463, 0.8849122877562188)
(18.133543, 0.7655430155404188)
(17.514101, 0.8872803273627611)
(25.06924, 0.8096546279062873)
(3.3066406, 0.778870301681877)
(2.8289104, 0.8982956952366206)
(1.10429, 0.8852266006226337)
(14.0651045, 0.7888071979573499)
(3.4950478, 0.8736074709147158)
(2.2948005, 0.7915915994271798)
(13.59112, 0.8405687163935449)
(1.816079, 0.7780594078827641)
(23.63976, 0.760122943511191)
(5.98189, 0.9384573040750467)
(15.615212, 0.6927479025739747)
(3.540135, 0.9346183880569262)
(4.7324557, 0.8765510927627551)
(12.189412, 0.8540545348671983)
(1.7883089, 0.8043317179597204)
(4.9531507, 0.9397450797283128)
(20.514269, 0.7454902538500654)
(17.259226, 0.7295069468062312)
(17.552479, 0.7583375094326437)
(4.8706346, 0.8272193118009417)
(16.38609, 0.8362224395845907)
(13.531786, 0.7711988406309072)
(6.9111114, 0.7895469649019794)
(15.765603, 0.7250301511446815)
(11.427302, 0.8765738711004848)
(13.24733, 0.87219239

(16.72945, 0.7316536690902816)
(4.451461, 0.8037912139165564)
(18.221922, 0.7922845448160609)
(26.036856, 0.7764214987775482)
(18.435495, 0.7214527141051768)
(14.668067, 0.7324912118443567)
(4.8034787, 0.89863423777497)
(0.2456996, 0.7997078166870338)
(14.896756, 0.7393921517130612)
(9.885102, 0.9369923112066715)
(12.001571, 0.8171475627525459)
(1.4111546, 0.9327434483214844)
(21.496338, 0.7519997580483284)
(2.7667737, 0.8789642032177426)
(8.292531, 0.9161774481864464)
(9.939715, 0.8670007395760524)
(8.958999, 0.8608456324419526)
(21.542812, 0.7446098097299652)
(0.7408228, 0.7181973356729626)
(22.16145, 0.754564151386993)
(1.4119127, 0.8367598858596086)
(9.644278, 0.7037834735002577)
(18.315638, 0.7500271898895539)
(1.2261145, 0.9132090913335352)
(11.581517, 0.7341833326142624)
(0.2616636, 0.8471469292408151)
(24.385921, 0.7669066885070341)
(6.155564, 0.7835773275560738)
(21.807375, 0.7994115754829625)
(14.577857, 0.8598021830729536)
(21.086184, 0.7488619142970643)
(1.2239454, 0.914864

Boosting also can be helpful to to such predictions

In [89]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Define number of estimators for Gradient Boosting
n_estimators = 100

# Initialize lists to store evaluation metrics
r2_scores = []
rmse_scores = []
Price_Booking521 = []
for i in range(filtered_df.shape[0]):
    # Extract prices and ratios
    transposed_prices = filtered_df[price_columns].iloc[i].T
    trans_ratios = filtered_df[ratios].iloc[i].T

    values_array = transposed_prices.to_numpy()
    v_arr = trans_ratios.to_numpy()

    pred_df = pd.DataFrame({
        'Prices': values_array.flatten(),
        'Ratios': v_arr.flatten()
    })

    X = pred_df[['Prices']]
    y = pred_df['Ratios']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize Gradient Boosting regression model
    model = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=0.1, random_state=42)

    # Fit the model
    model.fit(X_train, y_train)
    
    # Predict ratios for the price from df_bench
    price_start = df_bench[['Prices']].iloc[[i]] / 7
    #print(price_start)
    prediction_ = model.predict(price_start)
    #print(f"Predicted ratio for price: {prediction_[0]} and price is: {price_start}")
    pred_st = prediction_
    st = df_bench[['Prices']].iloc[[i]]
    #print(st)
    x = price_start.copy() 
    #print(f'price_start is {price_start}')
    
    ## This part needs to be cultivated
    while (x.values * 7 * prediction_ != st.values).all() and (x.values > 0).all():
        x -= 0.1 
        prediction_ = model.predict(x)
        #print(f'Price is: {x} pred ratio is: {prediction_} Start_price is: {price_start.values}')
    t = ()
    if (x.values < 0).all():
        t = (price_start.values[0][0], pred_st[0])
    else:
        t = (x.values[0][0], pred_st[0])
    print(t)
    Price_Booking521.append(t)
    # Evaluate the model using cross-validation
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    r2_scores_cv = cross_val_score(model, X, y, cv=kf, scoring='r2')
    r2_scores.append(np.mean(r2_scores_cv))
    mse_scores_cv = cross_val_score(model, X, y, cv=kf, scoring=make_scorer(mean_squared_error))
    rmse_scores.append(np.mean(np.sqrt(mse_scores_cv)))

# Print average evaluation metrics
print('Gradient Boosting Regression:')
print('Average R²:', np.mean(r2_scores))
print('Average RMSE:', np.mean(rmse_scores))


(7.0205674, 0.9310092747816393)
(3.1000857, 0.9483285260058898)
(18.556759, 0.8142235469331908)
(2.9955058, 0.9515553493788309)
(13.041038, 0.8218339772513035)
(2.9212766, 0.9008532962628001)
(14.541902, 0.8345132976589732)
(5.412914, 0.9177060645096902)
(14.172444, 0.8821839740972702)
(0.9719432, 0.958189237901894)
(1.1792558, 0.9502302748686637)
(4.3866625, 0.9582730118438003)
(24.654783, 0.8131668501939586)
(1.9987867, 0.9658413021255775)
(13.544943, 0.8660940608569165)
(12.282817, 0.836491497096012)
(16.554539, 0.8801701125345398)
(0.60571164, 0.9194109561457112)
(12.147621, 0.8265411944673121)
(1.6273702, 0.9400218912362454)
(0.25523123, 0.967785132887113)
(6.8815875, 0.9028500293916455)
(0.24396825, 0.8860885704576277)
(21.768536, 0.7813083822120078)
(0.29726687, 0.9892735871845916)
(1.7060547, 0.8979154027497621)
(11.397568, 0.9119331238088401)
(2.590768, 0.8962634492641599)
(16.730207, 0.7855220679646489)
(17.7598, 0.869571645556751)
(7.3101683, 0.8950752856299368)
(1.8504927, 

(16.753252, 0.8153834246557615)
(14.111824, 0.8155217773741001)
(6.432472, 0.9176843232514239)
(8.717521, 0.9110144427180925)
(3.0088875, 0.9604870101385973)
(5.0010753, 0.9356981752638743)
(7.9910097, 0.895815814267238)
(15.866678, 0.7700410586242004)
(1.5412205, 0.9396232505645699)
(18.962938, 0.7903005348309301)
(3.2979267, 0.9189811096011264)
(0.6355861, 0.9037841193676418)
(11.499339, 0.8797954981370948)
(1.5670357, 0.9162270870741278)
(20.12183, 0.8400828375633932)
(28.844343, 0.8366567598900058)
(17.68614, 0.8716010059976951)
(4.243194, 0.9378745003830545)
(25.372097, 0.817373502874504)
(20.463629, 0.8104559853176617)
(0.78022444, 0.9700699397292011)
(0.6647808, 0.8873952662275054)
(19.407675, 0.8173850590055172)
(3.924158, 0.9537984636375577)
(16.496593, 0.8808942789871052)
(2.9694078, 0.8864766228655181)
(10.3551, 0.8473309309693265)
(13.821711, 0.8545181917575085)
(9.5476675, 0.9171515380311267)
(12.839073, 0.852207709603643)
(0.43040377, 0.9880818401858245)
(14.526192, 0.775

(15.647751, 0.843151960136927)
(25.502752, 0.7971809313291927)
(20.567312, 0.7905752964847078)
(1.1518724, 0.8925439597388365)
(13.07018, 0.8863292638942188)
(11.988237, 0.8079516167229607)
(8.853732, 0.8889753945847995)
(25.30835, 0.8303134958925084)
(2.828558, 0.9671677589596495)
(1.621494, 0.9357234492233932)
(12.396071, 0.831753181121)
(17.857954, 0.8354527012399189)
(0.2555979, 0.9854566834067836)
(9.090911, 0.9300773257879913)
(6.8533664, 0.8992855954454091)
(10.072411, 0.8737928030348282)
(11.841455, 0.9205017866817693)
(22.056494, 0.774664826942064)
(16.944345, 0.8009562006022899)
(0.8132671, 0.9121033252157954)
(7.29889, 0.8991192639962319)
(11.82627, 0.8111076186152208)
(17.269165, 0.8737463999401041)
(7.1176567, 0.9095907454821585)
(12.913606, 0.8589948571640813)
(27.062912, 0.8052434253152418)
(17.64219, 0.8384115923440119)
(14.283051, 0.8864382526516464)
(24.98611, 0.7763225281883954)
(8.889357, 0.8802096472491728)
(10.622344, 0.8873551954813298)
(8.528059, 0.9192948161404

(10.75715, 0.8548371665762595)
(0.24395701, 0.8973234217104448)
(9.605018, 0.9238857265263907)
(17.504782, 0.8128658278070754)
(10.312797, 0.9048937093992762)
(19.370085, 0.7827110926353701)
(1.4881744, 0.9342594571142149)
(25.46384, 0.801376564788312)
(3.1190865, 0.8978484159240084)
(11.570112, 0.8739972436248069)
(12.962516, 0.8317798773470518)
(1.6061344, 0.8875988550641376)
(0.39128494, 0.9741069036411167)
(14.968195, 0.8381015298630705)
(5.8175993, 0.8994911132848761)
(19.00349, 0.8141700999323359)
(19.644428, 0.8153357130260424)
(2.5998495, 0.9612765436260423)
(5.9698224, 0.9162300763347525)
(8.006176, 0.9351482226121395)
(2.3165207, 0.890933662444079)
(12.849017, 0.882720521977785)
(0.24072461, 0.980076103943786)
(13.826081, 0.8010308972527527)
(21.679722, 0.8247114174426271)
(21.053177, 0.84322824010704)
(18.72851, 0.827468313349903)
(10.22256, 0.8790732233949246)
(12.322673, 0.801934441825874)
(4.078172, 0.9614185212992153)
(17.29761, 0.8203466064620704)
(19.890848, 0.84089768

(8.316198, 0.8850735324044056)
(12.428457, 0.8218211571905512)
(11.713539, 0.8086936209513338)
(29.798878, 0.7874552676601848)
(15.995384, 0.8434527906122473)
(5.296022, 0.9090486302410522)
(21.41048, 0.7995599776427054)
(11.209746, 0.8548211479095554)
(0.98489857, 0.978677638629428)
(26.456411, 0.7932485627238155)
(15.410499, 0.8256509213030776)
(16.150042, 0.8554463749362463)
(3.4855225, 0.9472303977124379)
(18.940216, 0.8626329704091437)
(23.095358, 0.7964720419920021)
(14.49727, 0.8690157068575705)
(6.9899793, 0.9173871120680303)
(6.4606357, 0.9433185452714178)
(0.35768676, 0.8850028722990714)
(10.422233, 0.8710059263503931)
(1.7020285, 0.9167022920935141)
(8.715435, 0.9420144088084347)
(10.299327, 0.934802365528137)
(15.605471, 0.8634726865974016)
(15.730789, 0.8505766321454873)
(16.05258, 0.8446940935303641)
(10.813436, 0.8687153960751465)
(14.749968, 0.8351152536001522)
(18.923616, 0.8590938528906965)
(16.716652, 0.8240921651049653)
(15.663079, 0.8333674929342682)
(14.653575, 0.

(11.829086, 0.8435084665774877)
(15.773623, 0.8699954554718418)
(24.0757, 0.7835084286088055)
(1.7355306, 0.8898516286901829)
(0.3140416, 0.9326627470893954)
(31.589499, 0.8101944194376833)
(15.765897, 0.798522440378895)
(2.4119153, 0.8997660347508714)
(24.67103, 0.82235812507594)
(1.0468403, 0.97618329138303)
(23.675581, 0.8097398346383055)
(8.530184, 0.8868138853754465)
(5.1245446, 0.9058810918202307)
(11.052005, 0.8805012355012868)
(19.113083, 0.8145847189137023)
(2.9138546, 0.9731589005120997)
(0.32027435, 0.9855841582258689)
(0.47789207, 0.8866632182484973)
(10.890528, 0.8804256453880847)
(4.314717, 0.9603423729303548)
(14.847635, 0.8219369670984383)
(21.879162, 0.8223037799292223)
(19.543009, 0.8185429344408622)
(18.705427, 0.8335849051861554)
(9.757523, 0.9123959282875939)
(21.516285, 0.8129330935401238)
(18.384432, 0.8233968701285044)
(16.971622, 0.7891165448146678)
(0.30578235, 0.889577191115194)
(2.4651427, 0.9652106189453467)
(22.272034, 0.7771241073033596)
(15.478365, 0.818

(13.2424755, 0.9148130511823177)
(16.44086, 0.8030656868111937)
(2.703324, 0.9352180686501348)
(19.916386, 0.7774841474067202)
(5.101139, 0.9486937829966277)
(17.942387, 0.8106168626978255)
(4.5076537, 0.923344018162973)
(12.989603, 0.8220761510938914)
(4.719528, 0.9365672434246384)
(12.33463, 0.9101402744995484)
(18.133543, 0.8483222811824449)
(17.514101, 0.9127672066270122)
(25.06924, 0.8123137209585981)
(3.3066406, 0.9514397018912134)
(2.8289104, 0.9090968068119013)
(1.10429, 0.9199170952075799)
(14.0651045, 0.8520627661831712)
(3.4950478, 0.8972996848290502)
(2.2948005, 0.9729034507657488)
(13.59112, 0.9020123277343931)
(1.816079, 0.9189519669580357)
(23.63976, 0.7931490428831355)
(5.98189, 0.9377904933086372)
(15.615212, 0.7838660748299527)
(3.540135, 0.9130410452638559)
(4.7324557, 0.9302202959452996)
(12.189412, 0.8757177223517595)
(1.7883089, 0.9214477811163191)
(4.9531507, 0.9647256714163269)
(20.514269, 0.7798846160439606)
(17.259226, 0.8061071683771754)
(17.552479, 0.7876660

(1.7280385, 0.8811789544737781)
(7.574113, 0.8987682961407127)
(17.267319, 0.8070297613526053)
(2.796497, 0.8853675204998446)
(1.6666546, 0.9611573693132355)
(16.115103, 0.857412848625449)
(6.3129, 0.9217141207832046)
(16.131433, 0.8192271502098059)
(16.72945, 0.7891888722348809)
(4.451461, 0.8956463836100466)
(18.221922, 0.8252228987503231)
(26.036856, 0.8031071505731587)
(18.435495, 0.7775236766902592)
(14.668067, 0.8109185840405438)
(4.8034787, 0.892555184717549)
(0.2456996, 0.898309808157424)
(14.896756, 0.8281202599602069)
(9.885102, 0.8977678910598246)
(12.001571, 0.8851233528618687)
(1.4111546, 0.9782561470549578)
(21.496338, 0.8135735216534125)
(2.7667737, 0.8971836639834188)
(8.292531, 0.9310207087635584)
(9.939715, 0.9195172221106973)
(8.958999, 0.9138655295263498)
(21.542812, 0.7842075344667552)
(0.7408228, 0.9348453044043737)
(22.16145, 0.8044148662471695)
(1.4119127, 0.8898181257439064)
(9.644278, 0.8779791359405014)
(18.315638, 0.7717113839874077)
(1.2261145, 0.9439124836

In [94]:
first_elements = [item[0] for item in Price_Booking521]
first_elements

[7.0205674,
 3.1000857,
 18.556759,
 2.9955058,
 13.041038,
 2.9212766,
 14.541902,
 5.412914,
 14.172444,
 0.9719432,
 1.1792558,
 4.3866625,
 24.654783,
 1.9987867,
 13.544943,
 12.282817,
 16.554539,
 0.60571164,
 12.147621,
 1.6273702,
 0.25523123,
 6.8815875,
 0.24396825,
 21.768536,
 0.29726687,
 1.7060547,
 11.397568,
 2.590768,
 16.730207,
 17.7598,
 7.3101683,
 1.8504927,
 23.193045,
 1.1462239,
 16.829012,
 16.598955,
 11.293541,
 6.7221246,
 12.449835,
 9.787645,
 19.787922,
 3.6176064,
 0.42261913,
 11.3163,
 13.946218,
 6.0581326,
 5.7531347,
 14.953123,
 11.104554,
 25.922098,
 10.936098,
 6.781283,
 7.5169554,
 2.8588505,
 20.220312,
 15.075484,
 15.349507,
 6.899054,
 6.5276575,
 3.5430834,
 8.640905,
 14.857667,
 0.25803798,
 4.613477,
 0.66284615,
 17.613894,
 19.737104,
 14.551742,
 3.7371948,
 0.5216448,
 1.151186,
 10.866999,
 8.245013,
 7.9128737,
 15.154577,
 10.989019,
 11.607323,
 14.366626,
 10.707498,
 10.572561,
 11.70105,
 13.564786,
 19.51646,
 1.477432,
 

In [95]:
price_booking_df = pd.DataFrame(first_elements, columns=['Price_Booking521'])

In [ ]:
price_booking_df.to_csv(output_file, index=False)